In [1]:
import pandas as pd
import dill
dill._dill._reverse_typemap['ClassType'] = type
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
#imputer
from sklearn.impute import SimpleImputer
from urllib import request, parse

import urllib.request
import json     

import sklearn.datasets

In [2]:
df = pd.read_csv('C:/Users/User/Desktop/Машинное обучение в бизнесе/курсовой проект/отток/customer_churn_dataset-master.csv')
df.head(10)

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0
5,8.0,51.0,Male,33.0,25.0,9.0,26.0,Premium,Annual,129.0,8.0,1.0
6,9.0,58.0,Female,49.0,12.0,3.0,16.0,Standard,Quarterly,821.0,24.0,1.0
7,10.0,55.0,Female,37.0,8.0,4.0,15.0,Premium,Annual,445.0,30.0,1.0
8,11.0,39.0,Male,12.0,5.0,7.0,4.0,Standard,Quarterly,969.0,13.0,1.0
9,12.0,64.0,Female,3.0,25.0,2.0,11.0,Standard,Quarterly,415.0,29.0,1.0


In [3]:
df = df.drop(columns = ['CustomerID'])

In [4]:
df.head(3)

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0


In [5]:
X = df.drop(['Churn'], axis=1)
y = df['Churn']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [7]:
X_test

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction
69437,27.0,Male,14.0,28.0,3.0,16.0,Standard,Monthly,862.00,9.0
224323,40.0,Male,19.0,2.0,8.0,28.0,Standard,Monthly,620.81,21.0
22398,27.0,Female,57.0,3.0,0.0,24.0,Standard,Annual,915.00,26.0
283708,40.0,Male,21.0,14.0,0.0,11.0,Basic,Annual,592.83,9.0
167619,39.0,Male,58.0,4.0,2.0,8.0,Standard,Monthly,694.00,15.0
...,...,...,...,...,...,...,...,...,...,...
101113,19.0,Female,48.0,10.0,3.0,22.0,Basic,Quarterly,344.00,4.0
425453,33.0,Male,40.0,28.0,1.0,9.0,Basic,Quarterly,582.14,23.0
432842,44.0,Male,33.0,14.0,2.0,18.0,Standard,Quarterly,855.02,5.0
437832,45.0,Female,7.0,9.0,2.0,11.0,Basic,Annual,613.61,7.0


In [8]:
# Категориальные признаки закодируем с помощью OneHotEncoding

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class Normalizer(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        
    def fit(self,X, y=None):
        return self
    
    def transform(self, X):
        scaler = MinMaxScaler()
        X[[self.key]] = scaler.fit_transform(X[[self.key]])
        return X[[self.key]]
        
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [9]:
categorical_columns = ['Gender', 'Subscription Type', 'Contract Length']
continuous_columns = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls', 'Payment Delay', 'Total Spend', 'Last Interaction']

In [10]:
#Теперь нам нужно под каждый признак создать трансформер и объединить их в список (сделаем это в цикле, чтобы не мучиться):
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('imputer', TextImputer(cat_col, '')),
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('imputer', TextImputer(cat_col, '')),
                ('selector', NumberSelector(key=cont_col)),
                ('Min_max', Normalizer(key=cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))

In [11]:
#Объединим все это в единый пайплайн:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [12]:
pipeline_xgb = Pipeline([
    ('features',feats),
    ('classifier', XGBClassifier(random_state = 12)),
])

In [13]:
#обучим:
pipeline_xgb.fit(X_train, y_train)

C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Gender',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Gender',
                                                                              value='')),
                                                                 ('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Subscription Type',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Subscription '
                                                            

In [14]:
with open("XGBClassifier_pipeline.dill", "wb") as f:
    dill.dump(pipeline_xgb, f)

### Проверка работоспособности и качества пайплайна

In [15]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [16]:
X_test.head(15)

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction
0,27.0,Male,14.0,28.0,3.0,16.0,Standard,Monthly,862.00,9.0
1,40.0,Male,19.0,2.0,8.0,28.0,Standard,Monthly,620.81,21.0
2,27.0,Female,57.0,3.0,0.0,24.0,Standard,Annual,915.00,26.0
3,40.0,Male,21.0,14.0,0.0,11.0,Basic,Annual,592.83,9.0
4,39.0,Male,58.0,4.0,2.0,8.0,Standard,Monthly,694.00,15.0
5,24.0,Male,49.0,29.0,2.0,5.0,Basic,Quarterly,992.95,17.0
6,35.0,Male,25.0,21.0,2.0,1.0,Premium,Quarterly,797.47,9.0
7,52.0,Female,50.0,22.0,10.0,4.0,Basic,Annual,187.00,9.0
8,41.0,Female,20.0,22.0,0.0,15.0,Basic,Monthly,814.00,6.0
9,22.0,Female,33.0,24.0,0.0,11.0,Standard,Quarterly,707.09,10.0


In [17]:
with open('XGBClassifier_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [18]:
predictions = pipeline.predict_proba(X_test.iloc[:400])
pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[self.key] = X[self.key].fillna(self.value)
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[self.key] = X[self.key].fillna(self.value)
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [19]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test.iloc[:400])

1.0

### Проверка того, что наше api работает

In [20]:
#!pip3 install flask-ngrok

In [21]:
from flask import Flask, request, jsonify
import pandas as pd

In [22]:
# Пробный запуск Flask

app = Flask(__name__)

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run(port=8082)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8082
Press CTRL+C to quit


### **Создаем сервис для обработки запросов к модели**

In [23]:
# Загружаем обученные модели
with open('XGBClassifier_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [24]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [31]:
# Обработчики и запуск Flask
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    #Age, Gender, Tenure, Usage_Frequency, Support_Calls, Payment_Delay, Subscription_Type, Contract_Length, Total_Spend, Last_Interaction = "", "", "","", "", "","", "","", ""
    
    request_json = request.get_json()
    
    if request_json["Age"]:
        Age = request_json['Age']
    
    if request_json["Gender"]:
        Gender = request_json['Gender']
                
    if request_json["Tenure"]:
        Tenure = request_json['Tenure']
        
    if request_json["Usage Frequency"]:
        Usage_Frequency = request_json['Usage Frequency']
    
    if request_json["Support Calls"]:
        Support_Calls = request_json['Support Calls']
                
    if request_json["Payment Delay"]:
        Payment_Delay = request_json['Payment Delay']
        
    if request_json["Subscription Type"]:
        Subscription_Type = request_json['Subscription Type']
    
    if request_json["Contract Length"]:
        Contract_Length = request_json['Contract Length']
                
    if request_json["Total Spend"]:
        Total_Spend = request_json['Total Spend']
    
    if request_json["Last Interaction"]:
        Last_Interaction = request_json['Last Interaction']
        
  
    preds = model.predict_proba(pd.DataFrame({'Age': [Age], 
                                              'Gender': [Gender],
                                              'Tenure': [Tenure],
                                              'Usage Frequency': [Usage_Frequency],
                                              'Support Calls': [Support_Calls],
                                              'Payment Delay': [Payment_Delay],
                                              'Subscription Type': [Subscription_Type],
                                              'Contract Length': [Contract_Length],
                                              'Total Spend': [Total_Spend],
                                              'Last Interaction': [Last_Interaction]}))
    data["predictions"] = preds[:, 1][0]
    data["predictions"] = data["predictions"].astype('str')
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run(port=8082)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8082
Press CTRL+C to quit
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a cop

OK


C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

OK
OK


...

OK
OK
OK
OK


C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

OK
OK


C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[[self.key]] = scaler.fit_transform(X[[self.key]])
C:\Users\User\AppData\Local\Temp\ipykernel_2848\3086940726.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

OK
OK
